In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import dspy
gpt3_turbo = dspy.OpenAI(model='gpt-3.5-turbo-1106', max_tokens=300)
dspy.configure(lm=gpt3_turbo)

/home/cch1006/langchain-startup/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Set up the trainset and the devset

In [3]:
# Create a larger training set
trainset = [
    dspy.Example(introduction="Tiger is a animal that is known for its strength and majestic stripes.", reason="I like tigers because they are powerful and have a regal presence."),
    dspy.Example(introduction="Lion is an animal that is often called the 'King of the Jungle' because of its imposing mane and roar.", reason="I like lions because they represent courage and leadership."),
    dspy.Example(introduction="Elephant is an animal that is the largest land mammal, known for its intelligence and long tusks.", reason="I like elephants because they are gentle giants with strong social bonds."),
    dspy.Example(introduction="Giraffe is an animal that is known for its long neck and legs, allowing it to reach leaves high in trees.", reason="I like giraffes because they are unique and graceful."),
    dspy.Example(introduction="Cheetah is an animal that is the fastest land animal, capable of reaching speeds up to 75 mph.", reason="I like cheetahs because they are incredibly swift and agile."),
    dspy.Example(introduction="Penguin is an animal that is known for its distinctive black and white plumage and ability to swim in cold waters.", reason="I like penguins because they are cute and resilient.")
]

# Create a development set
devset = [
    dspy.Example(introduction="Monkey is an animal that is known for its intelligence and ability to use tools.", reason="I like monkeys because they are playful and smart."),
    dspy.Example(introduction="Dog is an animal that is known for its loyalty and companionship to humans.", reason="I like dogs because they are faithful and bring joy to people's lives."),
    dspy.Example(introduction="Panda is an animal that is known for its distinctive black and white fur and love for bamboo.", reason="I like pandas because they are adorable and symbolize peace."),
    dspy.Example(introduction="Kangaroo is an animal that is known for its powerful hind legs and ability to hop at high speeds.", reason="I like kangaroos because they are fascinating and unique to Australia.")
]

print(trainset)
print(devset)

[Example({'introduction': 'Tiger is a animal that is known for its strength and majestic stripes.', 'reason': 'I like tigers because they are powerful and have a regal presence.'}) (input_keys=None), Example({'introduction': "Lion is an animal that is often called the 'King of the Jungle' because of its imposing mane and roar.", 'reason': 'I like lions because they represent courage and leadership.'}) (input_keys=None), Example({'introduction': 'Elephant is an animal that is the largest land mammal, known for its intelligence and long tusks.', 'reason': 'I like elephants because they are gentle giants with strong social bonds.'}) (input_keys=None), Example({'introduction': 'Giraffe is an animal that is known for its long neck and legs, allowing it to reach leaves high in trees.', 'reason': 'I like giraffes because they are unique and graceful.'}) (input_keys=None), Example({'introduction': 'Cheetah is an animal that is the fastest land animal, capable of reaching speeds up to 75 mph.',

#### Set up the metric , which is called `validate_answer` here

In [31]:
# Define the signature for automatic assessments.
class Assess(dspy.Signature):
    """Assess the quality of a tweet along the specified dimension."""

    assessed_text = dspy.InputField()
    assessment_question = dspy.InputField()
    assessment_answer = dspy.OutputField(desc="Yes or No")

gpt4T = dspy.OpenAI(model='gpt-4-1106-preview', max_tokens=1000, model_type='chat')

def validate_answer(example, pred, trace=None):
    # introduction_match = example.introduction.lower() in pred.reason.lower()
    reason_match = example.reason.lower() in pred.reason.lower()
    pattern_match = pred.reason.lower().startswith("i like")

    quality_question = "Is the assessed text a well-written and coherent response?"
    with dspy.context(lm=gpt4T):
        quality = dspy.Predict(Assess)(assessed_text=pred.reason, assessment_question=quality_question)

    quality_match = quality.assessment_answer.lower() == 'yes'

    print(reason_match, pattern_match, quality_match)
    if trace is None:  # if we're doing evaluation or optimization
        return (pattern_match + reason_match + quality_match) / 3.0
    else:  # if we're doing bootstrapping, i.e., self-generating good demonstrations of each step
        return pattern_match + reason_match and quality_match


#### Define a custom program that utilizes the ChainOfThought module to generate reasons:

In [28]:
class GenerateReason(dspy.Signature):
    """Based on the introduction, sum up the features what animals have.Take a assumption of what people could say when they like the animal.Start the sentence with 'I like' and end with a period."""

    introduction = dspy.InputField(desc="animal featurs and characteristics.")
    reason = dspy.OutputField(desc="The reason that I like the animal. Start the sentence with 'I like' and end with a period.")

In [29]:
class ReasonLikesAnimal(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_reason = dspy.ChainOfThought(GenerateReason)

    def forward(self, introduction):
        prediction = self.generate_reason(introduction=introduction)
        return dspy.Prediction(reason=prediction.reason)

program = ReasonLikesAnimal()
response = program(introduction=devset[0].introduction)
print(response)


Prediction(
    reason='I like monkeys because of their cleverness and adaptability.'
)


#### Specifying Input Keys

In [45]:
trainset_withinputs = []
for i in trainset:
    trainset_withinputs.append(i.with_inputs("introduction"))
print(trainset_withinputs)

devset_withinputs = []
for i in devset:
    devset_withinputs.append(i.with_inputs("introduction"))
print(devset_withinputs)

[Example({'introduction': 'Tiger is a animal that is known for its strength and majestic stripes.', 'reason': 'I like tigers because they are powerful and have a regal presence.'}) (input_keys={'introduction'}), Example({'introduction': "Lion is an animal that is often called the 'King of the Jungle' because of its imposing mane and roar.", 'reason': 'I like lions because they represent courage and leadership.'}) (input_keys={'introduction'}), Example({'introduction': 'Elephant is an animal that is the largest land mammal, known for its intelligence and long tusks.', 'reason': 'I like elephants because they are gentle giants with strong social bonds.'}) (input_keys={'introduction'}), Example({'introduction': 'Giraffe is an animal that is known for its long neck and legs, allowing it to reach leaves high in trees.', 'reason': 'I like giraffes because they are unique and graceful.'}) (input_keys={'introduction'}), Example({'introduction': 'Cheetah is an animal that is the fastest land an

#### DSPy Optimizer tuning using `BootstrapFewShot`

In [42]:
from dspy.teleprompt import BootstrapFewShot

config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

teleprompter = BootstrapFewShot(metric=validate_answer, **config)

optimized_cot = teleprompter.compile(ReasonLikesAnimal(), trainset=trainset_withinputs, valset=devset)

 17%|█▋        | 1/6 [00:01<00:08,  1.64s/it]

False True True


 33%|███▎      | 2/6 [00:06<00:13,  3.29s/it]

False True False


 50%|█████     | 3/6 [00:08<00:08,  2.87s/it]

False True True


 67%|██████▋   | 4/6 [00:10<00:04,  2.38s/it]

False True True


 83%|████████▎ | 5/6 [00:11<00:02,  2.39s/it]

False True True
Bootstrapped 4 full traces after 6 examples in round 0.


#### Evaluate its performance

In [46]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=devset_withinputs, metric=validate_answer, num_threads=1, display_progress=True, display_table=5)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

Average Metric: 0.6666666666666666 / 1  (66.7):  25%|██▌       | 1/4 [00:01<00:05,  1.83s/it]

False True True


Average Metric: 1.0 / 2  (50.0):  50%|█████     | 2/4 [00:05<00:06,  3.07s/it]               

False True False


Average Metric: 1.6666666666666665 / 3  (55.6):  75%|███████▌  | 3/4 [00:07<00:02,  2.37s/it]

...

Average Metric: 2.333333333333333 / 4  (58.3): 100%|██████████| 4/4 [00:09<00:00,  2.30s/it] 

False True True
Average Metric: 2.333333333333333 / 4  (58.3%)



/home/cch1006/langchain-startup/venv/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/home/cch1006/langchain-startup/venv/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:216: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.6666666666666666' '0.3333333333333333' '0.6666666666666666'
 '0.6666666666666666']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(lambda x: f'✔️ [{x}]' if x is True else f'{x}')


,introduction,example_reason,pred_reason,validate_answer
0,Monkey is an animal that is known for its intelligence and ability to use tools.,I like monkeys because they are playful and smart.,I like monkeys because they are clever and have impressive cognitive abilities.,0.6666666666666666
1,Dog is an animal that is known for its loyalty and companionship to humans.,I like dogs because they are faithful and bring joy to people's lives.,I like dogs because they are loyal and loving companions who have been by our side for centuries.,0.3333333333333333
2,Panda is an animal that is known for its distinctive black and white fur and love for bamboo.,I like pandas because they are adorable and symbolize peace.,I like pandas because they are cute and have a special diet that makes them stand out among other bears.,0.6666666666666666
3,Kangaroo is an animal that is known for its powerful hind legs and ability to hop at high speeds.,I like kangaroos because they are fascinating and unique to Australia.,I like kangaroos because they are fascinating creatures with powerful legs and a unique way of caring for their young.,0.6666666666666666


58.33

#### Inspect the Model's History

In [47]:
turbo.inspect_history(n=1)





Based on the introduction, sum up the features what animals have.Take a assumption of what people could say when they like the animal.Start the sentence with 'I like' and end with a period.

---

Follow the following format.

Introduction: animal featurs and characteristics.
Reasoning: Let's think step by step in order to ${produce the reason}. We ...
Reason: The reason that I like the animal. Start the sentence with 'I like' and end with a period.

---

Introduction: Tiger is a animal that is known for its strength and majestic stripes.
Reasoning: Let's think step by step in order to understand why tigers are so admired. We can start by looking at their powerful build and beautiful markings, which make them stand out in the animal kingdom.
Reason: I like tigers because they are fierce and beautiful creatures.

---

Introduction: Elephant is an animal that is the largest land mammal, known for its intelligence and long tusks.
Reasoning: Let's think step by step in order to understa

#### This is the untrained one, you can see that the `evaluate_compare(program)` is lower

In [53]:
# Set up the evaluator, which can be re-used in your code.
evaluate_compare = Evaluate(devset=devset_withinputs, metric=validate_answer, num_threads=1, display_progress=True, display_table=5)

# Launch evaluation.
evaluate_compare(program)

Average Metric: 2.0 / 4  (50.0): 100%|██████████| 4/4 [00:00<00:00, 440.08it/s]               

False True False
False True True
False True False
False True True
Average Metric: 2.0 / 4  (50.0%)



/home/cch1006/langchain-startup/venv/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/home/cch1006/langchain-startup/venv/lib/python3.10/site-packages/dspy/evaluate/evaluate.py:216: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.3333333333333333' '0.6666666666666666' '0.3333333333333333'
 '0.6666666666666666']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(lambda x: f'✔️ [{x}]' if x is True else f'{x}')


,introduction,example_reason,pred_reason,validate_answer
0,Monkey is an animal that is known for its intelligence and ability to use tools.,I like monkeys because they are playful and smart.,I like monkeys because of their cleverness and adaptability.,0.3333333333333333
1,Dog is an animal that is known for its loyalty and companionship to humans.,I like dogs because they are faithful and bring joy to people's lives.,"I like dogs because they are not only cute and cuddly, but also fiercely loyal and loving towards their human companions.",0.6666666666666666
2,Panda is an animal that is known for its distinctive black and white fur and love for bamboo.,I like pandas because they are adorable and symbolize peace.,I like pandas because they are adorable and have a peaceful nature.,0.3333333333333333
3,Kangaroo is an animal that is known for its powerful hind legs and ability to hop at high speeds.,I like kangaroos because they are fascinating and unique to Australia.,I like kangaroos because of their adorable pouches and their impressive jumping abilities.,0.6666666666666666


50.0